## Description

A tutorial to give an example on how to use AMIRAL and get the esitmated object. In here, we use a simulated image of VESTA as an example.

#### Package required for AMIRAL: 
- numpy
- matplotlib
- astropy
- maoppy --> but I need to think how to implement it because it is being set a bit differently
- decovbench --> 
- cython 

To implement the environment, import the environment from .yml file. (Check to see if it is the most-up-to-date version.)

Ideas from Romain: 
- Seperate AMIRAL from the PSF model because it can allow users to decide what model they would like to use
- For Romain's model (PSF), it is already in maoppy
- Might consider using *args (for parameters) and * * kwargs (for keywords) 
- __call__ () to implement function call operators (see here: https://stackoverflow.com/questions/9663562/what-is-the-difference-between-init-and-call) 

Note that:
- the shiftx and shifty in the aosystem is introduced because people might have different defintion of how to define a PSF (some uses a pixel but some uses 4 apparantly

# To-Do list
#### Clean up the notebook as there are some bugs!
<details>
  <summary>Click to expand!</summary>
  
  ## Gerenal
  - compare my psfao code with the maoppy (psfao model)
  - start reading the MISREAL code and see how to convert it
  - clean up my classes, i.e. those which need to be defined into class variable (outside of __init__ )or instance variable (define in __init__, unique to that instance)
    
  - try to put mu to zero for now and see how the minimiser behaves 
  - grab the otf, psf and the psd from IDL output and compare with python 
  - try scipy minimiser ...
</details>

# Section
<details>
  <summary>Click to expand!</summary>
  
  ## Heading
  1. [Generate a PSF](#Generate-a-PSF)
    * [Zero Padding](#Zero-Padding)
    * [Setting up the system](#Setting-up-the-system)
    * [Pupil Function](#Pupil-Function)
    
    
  2. [Convolution](#Convolution)
    
  3. [Debug](#Debug)
    
  3. [Minimisation](#Minimisation)
    * [Optimpack] (#Optimpack)
    * [SciPy] (#SciPy)
    
  4. [Gradient Test](#Gradient-Test)
  
</details>

In [ ]:
# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
#Change to your path
os.chdir("/Users/alau/Repo/amiral")
from amiral import instructment, utils, parameter, gradient, minimisation, array
from scipy.optimize import minimize 
%matplotlib inline

In [ ]:
# Global variable
# Parameters to modify
SEED = -1234       # Seed for random numbers generation
FLUX = 5e8         # Object total intensity [e-]
READ_OUT_NOISE = 10. # CCD read-out noise standard-deviation [e-]
DATA_DIR = "/Users/alau/IDLWorkspace/Data/Vesta_OASIS/"
DATA_FITS = DATA_DIR + "2018-06-08T05_27_05.809.fits"
DIMENSION = 256 # Dimension of the image
LOOP = True

#### To-do: add a file input which defines the telescope setup!

**Note**:
Both ```fft``` and ```ifft``` require the origin to be in the beginning of the vector (or in a 2D case, in the top-left of the array). If the input's centred like ```kx```, its origin must be shifted to the beginning with ```ifftshift```. 

To undo the change after transform, just ```ifftshift``` again :) 

## Generate a PSF 

aoSystem is used to provide a serveral outputs: PSD_array, pupil function (or the pupil plane), and ? - see the IDL output first! aoSystem inherits functions and methods from telescopeSetup (so I wont need to define all parameters again). 

#### Zero-Padding

- remember to pad before fft 
- zoom to area of interest after fft


First, we need to choose an image for deconvolution. In here, we have picked the asteriods. 

Consider changing the true asteriod into PSF $\circledast$ true object. 

#### Image formation theory
\begin{equation}
I = H*O+N, 
\end{equation}

where $I$ is the image, $H$ is the PSF, $O$ is the object and $N$ is the noise.


In the Fourier space,
\begin{equation}
    \hat{I} = \hat{O} \hat{H} + \hat{N}, 
\end{equation}
where $\hat{I}$, $\hat{O}$ and $\hat{N}$ are the Fourier transform of the image, object and noise respectively. Using the fourier space, it is easier for us to calculate the observed image, as we can take an inverse transform of $\hat{O} \hat{H} + \hat{N}$. 

In [ ]:
fits_img = fits.open(DATA_FITS)
fits_img.info() 

obj = fits_img[0].data
obj_resize = obj

obj_resize = array.scale_array(obj_resize, 4.)
obj_resize

Below is the object. 

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(121)
ax1.set_title('Original')
ax1.imshow(obj)

ax2 = fig.add_subplot(122)
ax2.set_title('Resized')
ax2.imshow(obj_resize)

Also, functions will be slowly mirgating from aoSystem to class PSF because I have not found out how to pass the class yet (to-be-honest --> it should not be hard so)


**Setting up the system**

In [ ]:
aosys = instructment.aoSystem(wavelength = 500e-9,resolution_rad = 3.5e-8
                              ,diameter = 7., occ_ratio = 0.1, no_acutuator = 30, dimension = DIMENSION)
fX, fY, freqnull = aosys.psd_frequency_array(DIMENSION, aosys.samp_factor[0])

In [ ]:
aosys.undersampling(0.3)

In [ ]:
plt.imshow(fX**2 + fY **2)
plt.show()

```print(help(aosys))``` is a printing the contents of a class. 
It is always useful for us to see what are in the class.

In [ ]:
print(help(aosys))

#### Pupil Function
To get the telescope component of the PSF, we need to know the pupil function first. From the pupil function, auto-correlation function of pupil function will give you the diffraction-limited OTF.

In [ ]:
pupil = aosys.get_pupil_plane()
plt.imshow(pupil)

In [ ]:
# As the functions requires 2D array, for 0 phase offset, 0*pupil_plane will keep the shape
# As you may know the whole otf is in the form of : h = h_tel + h_AO + h_shift (if we need one)
otf_tel = aosys.pupil_to_otf_tel(pupil)

fig, ax1 =  plt.subplots()
pos = ax1.imshow(np.real(otf_tel), cmap='Reds', interpolation='none')
fig.colorbar(pos, ax=ax1)

In [ ]:
psf_tel = np.fft.fftshift(np.real(utils.ifft2D(otf_tel)))

cx = len(psf_tel[0])/2
print(cx)

fig, ax1 =  plt.subplots()
pos = ax1.imshow(np.log10(psf_tel[256-80:256+80,256-80:256+80]), interpolation='none')
fig.colorbar(pos, ax=ax1)

In [ ]:
# Plotting the PSD of turbulent region (not corrected by the AO)
psd_halo = aosys.psd_residual_halo(r0=0.15)

fig, ax1 =  plt.subplots()
pos = ax1.imshow(np.log10(psd_halo), interpolation='none')
fig.colorbar(pos, ax=ax1)

#### Power Spectral Density (PSD)

Roddier1981 and Goodman1968 show that phase PSD consists of all information for describing our PSF.

In here, we seperated the PSD into 2 components: 
- $f <= f_{AO}$
- $f > f_{AO}$

$f_{AO}$ is the AO spatial cutoff frequency, which is the maximum spatial frequency of the phase to be corrected by the AO system.

To get the whole PSD, we need to combine those 2 components such that: 

\begin{equation}
    PSD_{total} = PSD_{AO} + PSD_{halo},
\end{equation}

where $PSD_{AO}$ describes AO-corrected frequencies regions and $PSD_{halo}$ refers to AO-uncorrected frequencies regions. 

#### PSD halo (AO-uncorrected region)

The uncorrected area is not modified by the AO system, hence, it follows the Kolmogorov law, 

\begin{equation}
    W_{\phi, Kolmo} (f) = 0.023r_0^{-5/3}f^{-11/3},
\end{equation}

where $f > f_{AO}$ and $r_0$ is the Fried parameter. We call this component $PSD_{halo}$ and it is **only** set by the knowledge of **Fried parameter**.

#### PSD AO (AO-corrected region)

Based on the moffat function, we are able to parameterise the AO-corrected PSD as:
\begin{equation}
    W_\phi(f) = \frac{\beta - 1}{\pi \alpha_x \alpha_y} \frac{M_A(f_x,f_y)}{1-(1+\frac{f_{AO}^2}{\alpha_x \alpha_y})^{1-\beta}}+ C, 
\end{equation}
where $f <= f_{AO}$, (description for parameters). 

In [ ]:
# Input parameter so we can have the moffat PSD
# param = [r0,bck,sig2,alpha,beta,theta,dx,dy]

## PSF parameter
PSF model is now made of a set of 7 parameters: {$\alpha_x,\alpha_y, \beta, \theta_R, C, r_0, A$}. (description for parameters)

In here, we consider **symmetric** case by setting $\alpha_x = \alpha_y$ and $\theta_R = 0$. Once the PSD and OTF of the telescopes are calculated, we get the PSF using: 
\begin{equation}
    h(\rho / \lambda) = \mathcal{F}^{-1} {h...}.
\end{equation}
(description of terms). 

In [ ]:
# To-do: compare this with the maoppy results
param= {
    "r0": 0.01,                  
    "background": 1e-10 ,      
    "amplitude": 3.0,       
    "ax": 0.05,                            
    "beta": 1.5,
    "mu": 1., 
    "rho0": 1., 
    "p": 1.
}
# IDL: moff[2,3] = a_x, a_y (which is not tested in the testing script)

In [ ]:
param_key, param_input = utils.dict2array(param)
psd_ao = aosys.psd_residual_ao (guess = param_input)

In [ ]:
# Plotting the PSD of AO region
fig, ax1 =  plt.subplots()
pos = ax1.imshow(np.log10(psd_ao), interpolation='none')
fig.colorbar(pos, ax=ax1)

In [ ]:
# Plotting the whole PSD 
psd = psd_halo + psd_ao 

fig, ax1 =  plt.subplots()
pos = ax1.imshow(np.log10(psd), interpolation='none')
fig.colorbar(pos, ax=ax1)

#### SR
SR is defined ...

In [ ]:
# Outputing the SR and the integral of PSD
integral, SR = aosys.psd_integral(psd, r0=param["r0"])

In [ ]:
otf_atmo = aosys.otf_atmo(psd)
plt.imshow(otf_atmo)

print(otf_atmo.shape)

In [ ]:
print(otf_tel.shape)


otf_total = aosys.otf_total(otf_tel, otf_atmo)

In [ ]:
psf_total = np.fft.fftshift(np.real(utils.ifft2D(otf_total)))
print(np.min(psf_total))


fig, ax =  plt.subplots(1,2)
fig.tight_layout()
pos = ax[0].imshow(np.log10(psf_total),vmin=-10)
ax[0].set_title('PSF total')
fig.colorbar(pos, ax=ax[0])

pos1 = ax[1].imshow(np.log10(psf_tel), vmin=-10)
ax[1].set_title('PSF telescope')
fig.colorbar(pos1, ax=ax[1])

plt.subplots_adjust(hspace=0.5)
plt.savefig("demo.png", dpi = 300)
plt.show()
# Need to look into - > Calculate the Strehl ratio (ratio of the max of 2 psfs)

In [ ]:
plt.imshow(np.log10(psf_total),vmin=-10, cmap = 'gray')

Since we have normalised the OTF and the PSF, the sum of PSF should be 1. and the maximum of the OTF should equal to 1. We can clearly see this when we plot the x-axis cut of the OTF. 

In [ ]:
print("\nSum of the PSF (which should be excatly 1.)", np.sum(np.abs(psf_total)))
print("\nMax of the otf_atmo: (which should be excatly 1.)", np.max(otf_atmo))
print("\nMax of the otf_tel: (which should be excatly 1.)", np.max(otf_tel))
print("\nMax of the OTF (which should be excatly 1.)", np.max(otf_total))

In [ ]:
# Strehl's ratio

psf_diffraction = aosys.psfao(otf_tel)
plt.imshow(np.log10(psf_diffraction))

print("\nSR (PSF ratio): ", np.max(psf_total)/ np.max(psf_diffraction))
print("\nSR from the integral: ", SR)

In [ ]:
rcParams['figure.figsize'] = 13 ,11
fig, ax = plt.subplots(1,3)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = int((DIMENSION*aosys.samp_factor[0])//2)

print(ycent)
ax[0].plot(np.abs(otf_total[ycent,...]))
ax[0].set_title('OTF(total)')
ax[0].axhline(y=1, color = 'r', ls = '--')

ax[1].plot(np.abs(otf_tel)[ycent, :])
ax[1].set_title('OTF(telescope)')
ax[1].axhline(y=1, color = 'r', ls = '--')

ax[2].plot(otf_atmo[ycent, :])
ax[2].set_title('OTF(atmosphere)')
ax[2].axhline(y=1, color = 'r', ls = '--')

In [ ]:
# For testing amiral script --> im = poidev(convolve(obj,psf)) + RON*randomn(SEED,Npix,Npix)
# which poidev is for generating poisson random deviate

## Convolution
Since we are using the true object in here, we need to convolve it with a PSF and add some noise before deconvolution. 

In here, we should adapt **same** formula and values from the IDL version. 


In IDL: 

symmetric: r0[m], bck sig2, alpha, beta 

non-sym: r0[m], bck sig2, alpha (x,y), theta, beta 

In [ ]:
aosys_dict = {
    'diameter': 7 , 
    'occ_ratio': 0.1 , 
    'no_acutuator' : 30, 
    'wavelength': 500, 
    'dimension': 256,
    'resolution_rad' : 3.5e-8 
}

aosys = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension']) 

In [ ]:
amiral_guess = {
    "r0": 0.2,                  
    "background": 1.5e-10 ,      
    "amplitude": 2.1,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

In [ ]:
fits_img = fits.open("/Users/alau/Data/amiral_fits/image_noise_1.fits")
fits_img.info() 

conv_image = fits_img[0].data 

## AMIRAL
In here, we start the main part of the amiral!

What you need for amiral: 
1. image you would like to process
2. PSF parameters and hyperparameters 
3. Bound for the PSF param and hyper param

We would like to fix some parameters: r0, amplitude, mu and rho0

In [ ]:
psf_param, psf_guess = utils.dict2array(amiral_guess)

amiralparam = parameter.amiralParam(img=conv_image, guess = psf_guess, aosys = aosys)

hyper_guess = amiralparam.hyperparam_initial(psf_guess, debug = True)
hyper_min, hyper_max = amiralparam.hyperparam_bound(psf_guess, p_upperbound = 100., debug = True)

psf_guess[-3] = hyper_guess[0]
psf_guess[-2] = hyper_guess[1]
psf_guess[-1] = hyper_guess[2]

param_min = np.asarray([0.01,0,0,1e-8,1.01])
param_max =  np.asarray([0.99,1e8,1e8,1e3,10])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

param_numerical_condition = np.array([1., 1e-4, 1., 1., 1.])
hyperparam_numerical_condition = np.array([hyper_guess[0], hyper_guess[1], 1.])

numerical_condition = np.concatenate((param_numerical_condition, hyperparam_numerical_condition))


# Note that I tried Boolean mask but it doesnt work ...
param_mask = np.array([1, 0, 1, 0, 0])
hyper_param_mask = np.array([1, 0, 0])

mask = np.concatenate((param_mask,hyper_param_mask))

#psf_guess = np.concatenate((psf_guess, hyper_guess))

amiral_cls = parameter.amiral(img=amiralparam.img, guess=psf_guess, aosys = aosys, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)

plt.imshow(amiral_cls.img[256-128:256+128, 256-128:256+128], cmap='gray')

parameter.amrial() is the child class of amiralParam. The intention of the having a parent class is for the methods for checking the parameter input. 

Here we set the initial condition for the amiral. 

#### Testing the numerical condition

So we are aware that numerical condition exists and the minimiser is not working so far ... Is it due to the numerical conditioning? 

Let us find out!

In [ ]:
print(psf_guess)
amiral_cls.varible4criterion(psf_guess)
amiral_cls.gradient(psf_guess)

In [ ]:
est_criterion, value_criterion, value_grad = amiral_cls.minimisation(psf_guess)

In [ ]:
print(est_criterion)

In [ ]:
# print(psf_guess)

In [ ]:
est_criterion, value_criterion, value_grad = amiral_cls.minimisation(est_criterion)

In [ ]:
print(est_criterion)

In [ ]:
# gradient_otf = aosys.gradient(psf_guess)

#### Critrtion map

Aim is to reproduce what has been plotted in the deconvolution paper. 

In [ ]:
# mu, rho0, p = amiral.hyperparam_initial(psf_guess)

# psf_guess[-3] = mu
# psf_guess[-2] = rho0 
# psf_guess[-1] = p

# print("\nInitial Hyperparameter:", mu, rho0, p)
    
# from scipy.optimize import Bounds 
    
# bounds = Bounds(ub = upperbound, lb = lowerbound)
# numeric_param = minimisation.param_physical2numerical(psf_guess, numerical_condition)
# res = minimize(amiral.marg_criterion, numeric_param, bounds = bounds)

In [ ]:
# print(res.x*numerical_condition)

In [ ]:
# psf_guess-res.x

In [ ]:
fwhm = 5

sigma = fwhm / (2. * np.sqrt(2*np.log(2)))
print(sigma)

lin = np.linspace(-0.5, 0.5, 256)
xx, yy = np.meshgrid(lin, lin)
gauss = (1/(2*np.pi * sigma **2)) * np.exp(-(np.square(xx) + np.square(yy)) / (2.*sigma**2))

plt.plot(gauss[128,:])






In [ ]:
gauss = utils.scale_array(gauss,2)
ft_gauss = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(gauss)))

In [ ]:
plt.imshow(np.real((ft_gauss))[256-20:256+20,256-20:256+20])

In [ ]:
plt.plot(gauss[128,:])

In [ ]:
from scipy import signal
test_dirac = 3*np.random.randn(512,512)+FLUX



test_impusle = signal.unit_impulse((512, 512), 'mid')*FLUX

ft_test_impusle = np.fft.fft2(test_impusle)

plt.imshow(test_impusle)

print(np.sum(test_impusle))

In [ ]:
ft_gauss = np.fft.fft2(gauss)


plt.imshow(np.abs(ft_gauss))

In [ ]:
star = np.fft.fftshift(np.fft.ifft2(otf_total*ft_test_impusle))

plt.imshow(np.log10(np.real(star[256-50:256+50,256-50:256+50])))


utils.info(star)
np.sum(np.real(star))

In [ ]:
from scipy.optimize import Bounds 

bounds = Bounds(ub = upperbound, lb = lowerbound)
numeric_param = minimisation.param_physical2numerical(test, amiral_cls.numerical_condition)
res = minimize(amiral_cls.marg_criterion, numeric_param, bounds = bounds)



In [ ]:
print(res.x)